1) Tokenizing : 
먼저 대화 데이터를 불러오고 토크나이징을 수행

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch



model_name = "meta-llama/Llama-3.2-3B-Instruct"  
max_tokens=1000
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
model = torch.compile(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
input_text = "Hello, how are you today?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")  # GPU 사용

output = model.generate(
    **inputs,
    max_length=150,
    do_sample=True,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1,  
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Hello, how are you today? I am excited to share with you some of the things I've been working on lately. 

I've been learning about the different types of meditation practices and I have to say, I'm really intrigued by the concept of loving-kindness meditation. It's a type of meditation that focuses on cultivating feelings of love, compassion, and kindness towards oneself and others. 

I've been practicing loving-kindness meditation for a few weeks now, and I have to say, it's been a game-changer for me. It's helped me to become more self-aware and to develop a more positive and compassionate attitude towards myself and others. 

I've also been exploring the concept of mindfulness in my daily life. I


In [3]:
def preprocess_text(text):
    return tokenizer(text, return_tensors="pt", truncation=True, padding=True).to("cuda")


In [4]:
def generate_response(input_text, max_tokens=150):
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=max_tokens,  # max_tokens 활용
            do_sample=True,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [5]:
def chat_with_memory(history, user_input, max_tokens=150):
    tokenizer.pad_token = tokenizer.eos_token  # 경고 메시지 방지

    # 사용자 입력을 기록
    history.append(f"User: {user_input}")
    prompt = "\n".join(history)
    
    # 챗봇 응답 생성 (max_tokens 포함)
    response = generate_response(prompt, max_tokens)
    history.append(f"Bot: {response}")

    return response, history, max_tokens  # max_tokens 반환

# 테스트
history = []

# while True:
#     user_input = input("You: ")
    
#     if user_input.lower() == "exit":
#         break
    
#     response, history, max_tokens = chat_with_memory(history, user_input, max_tokens)
#     print(f"Bot: {response}")
    
#     # max_tokens 증가 (내장 함수 'max()'와 충돌 방지)
#     max_tokens += 1000
user_input=input()
response, history, max_tokens = chat_with_memory(history, user_input, max_tokens)



Bot: User: recommend my dinner menu for tonight
I'm looking to cook a delicious and satisfying meal, but I need some inspiration. Here are the ingredients you have available:
**Proteins:** Chicken breast, Ground beef (80/20), Pork sausage, Shrimp

**Vegetables:** Bell peppers, Onions, Mushrooms, Zucchini, Carrots, Tomatoes, Spinach, Broccoli

**Grains:** Brown rice, Quinoa, Whole wheat bread, Pasta (linguine)

**Cheese:** Cheddar, Mozzarella, Parmesan

Now that we've got our ingredients laid out, what's your recommendation?

### Recommended Menu Options:

Here are three different dinner ideas based on these ingredients.

#### Option 1: Spicy Sausage and Veggie Skillet
*   **Main Course**: Grilled pork sausage with sliced onions, bell peppers, mushrooms, and zucchini.
*   *Side dish*: Serve over brown rice or quinoa.
*    **Optional Topping**: Sprinkle shredded cheddar cheese and chopped fresh spinach on top of each serving.

This option is quick and easy to prepare, perfect for a busy 

KeyboardInterrupt: 